In [19]:
import os
import pickle
import sys
import math
sys.path.append("../")
from od.env.config import SUMO_SIM_SECONDS,NET_TIMEOUT_SECONDS
from od.misc.statistic import NetFlowType
from od.network.types import ResourceAllocatorType,BaseStationType
from od.misc.interest import InterestConfig
from od.env.config import SUMO_SIM_STEPS,ROOT_DIR
from od.env.station import BS_PRESET
from od.social.manager.types import DynamicSocialGroupBehaviour
import od.misc.logger
import od.misc.statistic as oms
from datetime import datetime

In [20]:
class FakeBaseStation:
    def __init__(self,name,pos,bs_type,serial):
        self.name = name
        self.pos = pos
        self.type = bs_type
        self.serial = serial
    def __repr__(self) -> str:
        return self.name
    def __index__(self):
        return self.serial

In [21]:
def Run(statistic,interest_config):
    statistic.nft_traffic = {}
    statistic.raw_sg_header = None
    statistic.interest_config = interest_config
    statistic.dirpath = "../data/OLD/GS20/" + statistic.interest_config.folder()
    print(statistic.dirpath)
    BS_CONFIG =  {
            name:bs_config 
            for name,bs_config in BS_PRESET.items()
            if (
                bs_config["type"] == BaseStationType.UMA  or 
                (
                    interest_config.req_rsu and 
                    bs_config["type"] == BaseStationType.UMI
                )
            )
    }  
    time_text = datetime.now().strftime("%Y-%m-%d %H-%M-%S")
    class GV:
        NET_STATION_CONTROLLER = [ 
            FakeBaseStation(
                name,
                BS_CONFIG[name]["pos"],
                BS_CONFIG[name]["type"] ,
                i
            )   for i,name in enumerate(BS_CONFIG)
        ]
        STATISTIC = od.misc.logger.Logger(statistic.dirpath,"Statistic ({}).txt".format(time_text))
        RESULT = od.misc.logger.Logger(statistic.dirpath,"Result ({}).txt".format(time_text))
    oms.GV = GV
    # preprocess overtimes
    for sg in statistic.social_group:
        for record in statistic.sg_header[sg].values():
            # ignore appdata that're flagged overtime.
            if(record.is_src_ot):
                continue
            else:
                # Check data source overtime condition.
                in_txq_time = list(filter(lambda x: len(x) > 0, record.time_bs_txq))
                min_enter_txq_time = min(
                    list(map(
                        lambda x: x[0][0],
                        in_txq_time
                    )) + [NET_TIMEOUT_SECONDS + record.at]
                )
                if(len(in_txq_time) == 0 or
                    min_enter_txq_time - record.at >= NET_TIMEOUT_SECONDS):
                    record.is_src_ot = True
                else:
                    # Check base station overtime condition.
                    for bs in GV.NET_STATION_CONTROLLER:
                        if(
                            len(record.time_bs_txq[bs]) > 0 and
                            record.time_bs_txq[bs][-1][1] - record.at >= NET_TIMEOUT_SECONDS
                        ):
                            record.is_bs_ot[bs] = True
    statistic.Report(True)
    GV.STATISTIC.Encapsulate()
    GV.RESULT.Encapsulate()
    

In [23]:
for res_alloc_type in [ResourceAllocatorType.NOMA_APR]:
    for rsu in [False, True]:
        for qos_re_class in [True, False] if rsu else [False]:
            for traffic_scale in [i / 10 for i in range(10, 15, 1)]:
                for seed in [i + 1 for i in range(9)]:
                    interest_config = InterestConfig(DynamicSocialGroupBehaviour.MAX_N_MEMBER,20, qos_re_class, res_alloc_type, rsu, traffic_scale, seed)
                    old_statistic = None
                    with open("../data/GS20/{}/object.pickle".format(interest_config.folder_legacy(1)), "rb") as file:
                        old_statistic = pickle.load(file)
                    Run(old_statistic, interest_config)


../data/OLD/GS20/MAX_N_MEMBER(20)/nQoS/nRSU/NOMA_APR/D1.0/S1/
Skipping Preprocess, Since there's no Raw Data.
../data/OLD/GS20/MAX_N_MEMBER(20)/nQoS/nRSU/NOMA_APR/D1.0/S2/
Skipping Preprocess, Since there's no Raw Data.
../data/OLD/GS20/MAX_N_MEMBER(20)/nQoS/nRSU/NOMA_APR/D1.0/S3/
Skipping Preprocess, Since there's no Raw Data.
../data/OLD/GS20/MAX_N_MEMBER(20)/nQoS/nRSU/NOMA_APR/D1.0/S4/
Skipping Preprocess, Since there's no Raw Data.
../data/OLD/GS20/MAX_N_MEMBER(20)/nQoS/nRSU/NOMA_APR/D1.0/S5/
Skipping Preprocess, Since there's no Raw Data.
../data/OLD/GS20/MAX_N_MEMBER(20)/nQoS/nRSU/NOMA_APR/D1.0/S6/
Skipping Preprocess, Since there's no Raw Data.
../data/OLD/GS20/MAX_N_MEMBER(20)/nQoS/nRSU/NOMA_APR/D1.0/S7/
Skipping Preprocess, Since there's no Raw Data.
../data/OLD/GS20/MAX_N_MEMBER(20)/nQoS/nRSU/NOMA_APR/D1.0/S8/
Skipping Preprocess, Since there's no Raw Data.
../data/OLD/GS20/MAX_N_MEMBER(20)/nQoS/nRSU/NOMA_APR/D1.0/S9/
Skipping Preprocess, Since there's no Raw Data.
../data/OL